In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary
from torchmetrics import Accuracy
from torchvision import datasets
from torchvision.transforms import ToTensor
import dagshub

import mlflow

In [8]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [9]:
print(f"Image size: {training_data[0][0].shape}")
print(f"Size of training dataset: {len(training_data)}")
print(f"Size of test dataset: {len(test_data)}")

Image size: torch.Size([1, 28, 28])
Size of training dataset: 60000
Size of test dataset: 10000


In [10]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [11]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(8, 16, kernel_size=3),
            nn.ReLU(),
            nn.Flatten(),
            nn.LazyLinear(10),  # 10 classes in total.
        )

    def forward(self, x):
        return self.model(x)

In [12]:
dagshub.init(repo_owner='mohannadrateb',repo_name="MLflow", mlflow=True)
mlflow.set_tracking_uri("https://dagshub.com/mohannadrateb/MLflow.mlflow")

Initialized MLflow to track repo "mohannadrateb/MLflow"

Repository mohannadrateb/MLflow initialized!

In [13]:
mlflow.set_experiment("/mlflow-pytorch-quickstart")

<Experiment: artifact_location='mlflow-artifacts:/21cb4ea6d81e40b697ecea5361fbee50', creation_time=1720869156240, experiment_id='1', last_update_time=1720869156240, lifecycle_stage='active', name='/mlflow-pytorch-quickstart', tags={}>

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [15]:
def train(dataloader, model, loss_fn, metrics_fn, optimizer, epoch):
    """Train the model on a single pass of the dataloader.

    Args:
        dataloader: an instance of `torch.utils.data.DataLoader`, containing the training data.
        model: an instance of `torch.nn.Module`, the model to be trained.
        loss_fn: a callable, the loss function.
        metrics_fn: a callable, the metrics function.
        optimizer: an instance of `torch.optim.Optimizer`, the optimizer used for training.
        epoch: an integer, the current epoch number.
    """
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)
        accuracy = metrics_fn(pred, y)

        # Backpropagation.
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch
            step = batch // 100 * (epoch + 1)
            mlflow.log_metric("loss", f"{loss:2f}", step=step)
            mlflow.log_metric("accuracy", f"{accuracy:2f}", step=step)
            print(f"loss: {loss:2f} accuracy: {accuracy:2f} [{current} / {len(dataloader)}]")

In [16]:
def evaluate(dataloader, model, loss_fn, metrics_fn, epoch):
    """Evaluate the model on a single pass of the dataloader.

    Args:
        dataloader: an instance of `torch.utils.data.DataLoader`, containing the eval data.
        model: an instance of `torch.nn.Module`, the model to be trained.
        loss_fn: a callable, the loss function.
        metrics_fn: a callable, the metrics function.
        epoch: an integer, the current epoch number.
    """
    num_batches = len(dataloader)
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            eval_loss += loss_fn(pred, y).item()
            eval_accuracy += metrics_fn(pred, y)

    eval_loss /= num_batches
    eval_accuracy /= num_batches
    mlflow.log_metric("eval_loss", f"{eval_loss:2f}", step=epoch)
    mlflow.log_metric("eval_accuracy", f"{eval_accuracy:2f}", step=epoch)

    print(f"Eval metrics: \nAccuracy: {eval_accuracy:.2f}, Avg loss: {eval_loss:2f} \n")

In [17]:
epochs = 3
loss_fn = nn.CrossEntropyLoss()
metric_fn = Accuracy(task="multiclass", num_classes=10).to(device)
model = ImageClassifier().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

/Users/mohannadrateb/Desktop/portfolio/Port_projects/Data-Science-Eng-Portfolio/MLFLOW/LLM_Evaluate/.venv/lib/python3.9/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [18]:
with mlflow.start_run() as run:
    params = {
        "epochs": epochs,
        "learning_rate": 1e-3,
        "batch_size": 64,
        "loss_function": loss_fn.__class__.__name__,
        "metric_function": metric_fn.__class__.__name__,
        "optimizer": "SGD",
    }
    # Log training parameters.
    mlflow.log_params(params)

    # Log model summary.
    with open("model_summary.txt", "w") as f:
        f.write(str(summary(model)))
    mlflow.log_artifact("model_summary.txt")

    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, metric_fn, optimizer, epoch=t)
        evaluate(test_dataloader, model, loss_fn, metric_fn, epoch=0)

    # Save the trained model to MLflow.
    mlflow.pytorch.log_model(model, "model")

Epoch 1
-------------------------------
loss: 2.297716 accuracy: 0.125000 [0 / 938]
loss: 2.218623 accuracy: 0.390625 [100 / 938]
loss: 1.992512 accuracy: 0.437500 [200 / 938]
loss: 1.692949 accuracy: 0.625000 [300 / 938]
loss: 1.170610 accuracy: 0.671875 [400 / 938]
loss: 0.989121 accuracy: 0.687500 [500 / 938]
loss: 0.921422 accuracy: 0.703125 [600 / 938]
loss: 0.752936 accuracy: 0.765625 [700 / 938]
loss: 0.786248 accuracy: 0.718750 [800 / 938]
loss: 0.811922 accuracy: 0.718750 [900 / 938]
Eval metrics: 
Accuracy: 0.74, Avg loss: 0.737757 

Epoch 2
-------------------------------
loss: 0.688798 accuracy: 0.812500 [0 / 938]
loss: 0.779676 accuracy: 0.687500 [100 / 938]
loss: 0.502168 accuracy: 0.812500 [200 / 938]
loss: 0.770441 accuracy: 0.734375 [300 / 938]
loss: 0.707769 accuracy: 0.640625 [400 / 938]
loss: 0.682001 accuracy: 0.781250 [500 / 938]
loss: 0.714588 accuracy: 0.703125 [600 / 938]
loss: 0.637330 accuracy: 0.781250 [700 / 938]
loss: 0.683414 accuracy: 0.718750 [800 / 938

In [19]:
logged_model = f"runs:/{run.info.run_id}/model"
loaded_model = mlflow.pyfunc.load_model(logged_model)

/Users/mohannadrateb/Desktop/portfolio/Port_projects/Data-Science-Eng-Portfolio/MLFLOW/LLM_Evaluate/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
outputs = loaded_model.predict(training_data[0][0][None, :].numpy())